In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 104.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 5.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.31.0 which is incompatible.


In [ ]:
!pip install -q \
    transformers==4.28.1 \
    trl==0.4.1 \
    peft==0.4.0 \
    accelerate==0.21.0 \
    datasets \
    sentencepiece \
    huggingface_hub


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.28.1 which is incompatible.


In [ ]:
# libraries
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer




/usr/local/lib/python3.11/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


/usr/local/lib/python3.11/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


ModuleNotFoundError: No module named 'triton.ops'

In [ ]:
!pip uninstall -y bitsandbytes


In [ ]:
!pip install -q \
    transformers==4.28.1 \
    trl==0.4.1 \
    peft==0.4.0 \
    accelerate==0.21.0 \
    datasets \
    sentencepiece \
    huggingface_hub


In [ ]:
import torch
torch.cuda.is_available()


False

In [ ]:
model_name = "NousResearch/Llama-2-7b-hf"

from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ValueError: Need either a `state_dict` or a `save_folder` containing offloaded weights.

In [ ]:
def load_json_lines_safe(path):
    import json

    cleaned_data = []
    with open(path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f, 1):
            line = line.strip()
            if not line:
                continue
            try:
                cleaned_data.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Skipping line {i}: {e}")
                continue
    return cleaned_data

# Load
train_data = load_json_lines_safe("/content/hin_train.json")
valid_data = load_json_lines_safe("/content/hin_valid.json")
test_data  = load_json_lines_safe("/content/hin_test.json")

#  Dataset
from datasets import Dataset, DatasetDict
dataset = DatasetDict({
    "train": Dataset.from_list(train_data),
    "validation": Dataset.from_list(valid_data),
    "test": Dataset.from_list(test_data)
})


dataset["train"][0]


{'unique_identifier': 'hin1',
 'native word': 'जन्मदिवस',
 'english word': 'janamdivas',
 'source': 'Dakshina',
 'score': None}

In [ ]:
def merge_inputs(example):
    return {
        "text": f"Transliterate to Hindi: {example['english word']}\nOutput: {example['native word']}"
    }

formatted_dataset = dataset.map(merge_inputs, remove_columns=["unique_identifier", "native word", "english word", "source"])


Map:   0%|          | 0/1299155 [00:00<?, ? examples/s]

Map:   0%|          | 0/6357 [00:00<?, ? examples/s]

Map:   0%|          | 0/10112 [00:00<?, ? examples/s]

In [ ]:
formatted_dataset["train"][0]


{'score': None, 'text': 'Transliterate to Hindi: janamdivas\nOutput: जन्मदिवस'}

In [ ]:
def tokenize(example):
    return tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

tokenized_dataset = formatted_dataset.map(tokenize, batched=True)

Map:   0%|          | 0/1299155 [00:00<?, ? examples/s]

Map:   0%|          | 0/6357 [00:00<?, ? examples/s]

Map:   0%|          | 0/10112 [00:00<?, ? examples/s]

In [ ]:
from peft import LoraConfig
from transformers import TrainingArguments

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

training_args = TrainingArguments(
    output_dir="./llama-peft-output",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    logging_steps=10,
    learning_rate=2e-4,
    fp16=True,
    push_to_hub=False,
    report_to="none",
    save_total_limit=2
)


ModuleNotFoundError: No module named 'triton.ops'

In [ ]:
peft_model.gradient_checkpointing_enable()

In [ ]:
from peft import get_peft_model

peft_model = get_peft_model(model, lora_config)


In [ ]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

bnb_config = BitsAndBytesConfig(load_in_4bit=True)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at TinyLlama/TinyLlama-1.1B-Chat-v1.0 and are newly initialized: ['model.layers.10.self_attn.rotary_emb.inv_freq', 'model.layers.6.self_attn.rotary_emb.inv_freq', 'model.layers.21.self_attn.rotary_emb.inv_freq', 'model.layers.18.self_attn.rotary_emb.inv_freq', 'model.layers.0.self_attn.rotary_emb.inv_freq', 'model.layers.19.self_attn.rotary_emb.inv_freq', 'model.layers.12.self_attn.rotary_emb.inv_freq', 'model.layers.8.self_attn.rotary_emb.inv_freq', 'model.layers.7.self_attn.rotary_emb.inv_freq', 'model.layers.9.self_attn.rotary_emb.inv_freq', 'model.layers.17.self_attn.rotary_emb.inv_freq', 'model.layers.2.self_attn.rotary_emb.inv_freq', 'model.layers.14.self_attn.rotary_emb.inv_freq', 'model.layers.4.self_attn.rotary_emb.inv_freq', 'model.layers.1.self_attn.rotary_emb.inv_freq', 'model.layers.5.self_attn.rotary_emb.inv_freq', 'model.layers.16.self_attn.rotary_emb.inv_freq', 'model.layers.13.self_attn.rot

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
